In [2]:
import pandas as pd

df = pd.read_csv("funnel_events.csv")

print("\nRows:", df.shape[0])
print("Users:", df["user_id"].nunique())
print("\nEvent counts:")
print(df["event_name"].value_counts())



Rows: 24649
Users: 10000

Event counts:
event_name
visit              10000
product_view        7004
add_to_cart         3846
checkout            2318
payment_success     1481
Name: count, dtype: int64


In [3]:
df.head()

,user_id,event_time,event_name,device_type,traffic_source,product_category,price,funnel_success
0,1,2026-01-01 06:30:57,visit,Mobile,Referral,Fashion,1412,No
1,1,2026-01-01 06:31:57,product_view,Mobile,Referral,Fashion,1412,No
2,2,2026-01-05 22:16:16,visit,Mobile,Social Media,Electronics,4967,No
3,2,2026-01-05 22:18:16,product_view,Mobile,Social Media,Electronics,4967,No
4,2,2026-01-05 22:23:16,add_to_cart,Mobile,Social Media,Electronics,4967,No


In [4]:
# Funnel Stage Order
funnel_order = {
    "visit": 1,
    "product_view": 2,
    "add_to_cart": 3,
    "checkout": 4,
    "payment_success": 5
}
df["event_rank"] = df["event_name"].map(funnel_order)


In [5]:
df.head()

,user_id,event_time,event_name,device_type,traffic_source,product_category,price,funnel_success,event_rank
0,1,2026-01-01 06:30:57,visit,Mobile,Referral,Fashion,1412,No,1
1,1,2026-01-01 06:31:57,product_view,Mobile,Referral,Fashion,1412,No,2
2,2,2026-01-05 22:16:16,visit,Mobile,Social Media,Electronics,4967,No,1
3,2,2026-01-05 22:18:16,product_view,Mobile,Social Media,Electronics,4967,No,2
4,2,2026-01-05 22:23:16,add_to_cart,Mobile,Social Media,Electronics,4967,No,3


In [16]:
# Event Order Validation - “Did users follow the correct funnel steps in order?”
invalid_users = (
    df.sort_values(["user_id", "event_time"])
      .groupby("user_id")["event_rank"]
      .apply(lambda x: not x.is_monotonic_increasing)
)

print("Users with invalid event order:", invalid_users.sum())

Users with invalid event order: 0


validated event order to ensure users followed the correct funnel sequence, and no invalid user journeys were found

In [7]:
funnel_counts = (
    df.groupby("event_name")["user_id"]
      .nunique()
      .reset_index(name="users")
      .sort_values("users", ascending=False)
)

print(funnel_counts)

        event_name  users
4            visit  10000
3     product_view   7004
0      add_to_cart   3846
1         checkout   2318
2  payment_success   1481


In [8]:

funnel_counts = funnel_counts.sort_values("users", ascending=False).reset_index(drop=True)

funnel_counts["conversion_rate_%"] = (
    funnel_counts["users"] / funnel_counts["users"].shift(1) * 100
).round(2)

funnel_counts["drop_off_%"] = (100 - funnel_counts["conversion_rate_%"]).round(2)

print(funnel_counts)


        event_name  users  conversion_rate_%  drop_off_%
0            visit  10000                NaN         NaN
1     product_view   7004              70.04       29.96
2      add_to_cart   3846              54.91       45.09
3         checkout   2318              60.27       39.73
4  payment_success   1481              63.89       36.11


Identified major funnel drop-off between product view and add-to-cart stage, indicating potential UX or pricing friction.

In [13]:
# Device-Wise Funnel Conversion - "Are mobile users less likely to complete the funnel?"
device_funnel = (
    df.groupby(["device_type", "event_name"])["user_id"]
      .nunique()
      .reset_index(name="users")
)
device_funnel["event_rank"] = device_funnel["event_name"].map(funnel_order)
device_funnel = device_funnel.sort_values(
    ["device_type", "event_rank"]
)

device_funnel["conversion_rate_%"] = (
    device_funnel.groupby("device_type")["users"]
    .pct_change() * 100
).round(2)

print("\nDevice-wise Funnel:")
print(device_funnel)


Device-wise Funnel:
  device_type       event_name  users  event_rank  conversion_rate_%
4     Desktop            visit   5073           1                NaN
3     Desktop     product_view   3553           2             -29.96
0     Desktop      add_to_cart   1965           3             -44.69
1     Desktop         checkout   1171           4             -40.41
2     Desktop  payment_success    746           5             -36.29
9      Mobile            visit   4927           1                NaN
8      Mobile     product_view   3451           2             -29.96
5      Mobile      add_to_cart   1881           3             -45.49
6      Mobile         checkout   1147           4             -39.02
7      Mobile  payment_success    735           5             -35.92


Mobile and Desktop users behave almost the same in the funnel. The biggest user drop happens at the checkout stage, which may indicate issues like payment or checkout experience.

In [14]:
# Traffic Source Drop-Off - “Which acquisition channels bring low-intent users?”
traffic_funnel = (
    df.groupby(["traffic_source", "event_name"])["user_id"]
      .nunique()
      .reset_index(name="users")
)
traffic_funnel["event_rank"] = traffic_funnel["event_name"].map(funnel_order)

traffic_funnel = traffic_funnel.sort_values(
    ["traffic_source", "event_rank"]
)

traffic_funnel["conversion_rate_%"] = (
    traffic_funnel.groupby("traffic_source")["users"]
    .pct_change() * 100
).round(2)

print("\nTraffic Source Funnel:")
print(traffic_funnel)


Traffic Source Funnel:
   traffic_source       event_name  users  event_rank  conversion_rate_%
4             Ads            visit   2500           1                NaN
3             Ads     product_view   1793           2             -28.28
0             Ads      add_to_cart   1012           3             -43.56
1             Ads         checkout    608           4             -39.92
2             Ads  payment_success    382           5             -37.17
9         Organic            visit   2507           1                NaN
8         Organic     product_view   1721           2             -31.35
5         Organic      add_to_cart    933           3             -45.79
6         Organic         checkout    545           4             -41.59
7         Organic  payment_success    347           5             -36.33
14       Referral            visit   2448           1                NaN
13       Referral     product_view   1708           2             -30.23
10       Referral      add_

Users coming from referrals are more likely to complete purchases. Users from Ads and Organic sources drop more often, so these channels may need better targeting or messaging.

In [15]:
# Category-Wise Drop-Off - “Which product categories perform best?”
category_funnel = (
    df.groupby(["product_category", "event_name"])["user_id"]
      .nunique()
      .reset_index(name="users")
)

category_funnel["event_rank"] = category_funnel["event_name"].map(funnel_order)

category_funnel = category_funnel.sort_values(
    ["product_category", "event_rank"]
)

category_funnel["conversion_rate_%"] = (
    category_funnel.groupby("product_category")["users"]
    .pct_change() * 100
).round(2)

print("\nCategory-wise Funnel:")
print(category_funnel)


Category-wise Funnel:
   product_category       event_name  users  event_rank  conversion_rate_%
4            Beauty            visit   2055           1                NaN
3            Beauty     product_view   1453           2             -29.29
0            Beauty      add_to_cart    810           3             -44.25
1            Beauty         checkout    503           4             -37.90
2            Beauty  payment_success    324           5             -35.59
9       Electronics            visit   1996           1                NaN
8       Electronics     product_view   1391           2             -30.31
5       Electronics      add_to_cart    779           3             -44.00
6       Electronics         checkout    470           4             -39.67
7       Electronics  payment_success    294           5             -37.45
14          Fashion            visit   2019           1                NaN
13          Fashion     product_view   1400           2             -30.66
10

Fashion products convert better than other categories. Sports and Home products see more users dropping off, which may be due to higher prices or longer decision time.